# 6. Практика

Итак, вы познакомились с основными методами построения рекомендательных систем, и теперь настало время закрепить полученные знания на практике. В предыдущем модуле мы начали строить РС для сервиса чтения статей CI&T DeskDrop. В этом юните мы продолжим работу над ней.

Примечание. Если у вас не сохранился код, который мы использовали ранее, вы можете найти его в ноутбуке.
Так как теперь вам известны более сложные алгоритмы построения рекомендательных систем, начнём с них.

Для начала необходимо построить матрицу, в которой по столбцам будут находиться id статей, по строкам — id пользователей, а на пересечениях строк и столбцов — оценка взаимодействия пользователя со статьёй. Если взаимодействия не было, в соответствующей ячейке должен стоять ноль.

Для начала загрузим датасет ["Articles sharing and reading from CI&T DeskDrop"](https://www.kaggle.com/datasets/gspmoreira/articles-sharing-reading-from-cit-deskdrop), включающий в себя собранные за один год логи DeskDrop — платформы для внутренних коммуникаций, разработанной CI&T и ориентированной на компании, использующие Google Workspace (Google G Suite). Среди прочего, эта платформа позволяет сотрудникам компаний делиться актуальными статьями со своими коллегами.

В датасете содержится около 73 тысяч записей о взаимодействии пользователей с более чем тремя тысячами публичных статей, размещённых на платформе.

Информация в наборе данных:

Оригинальный URL, название и текст статьи.

Контекст посещений пользователей, например дата/время, клиент (мобильное приложение/браузер) и геолокация.

Различные типы взаимодействия, что позволяет сделать вывод об уровне заинтересованности пользователя в статьях, например комментарии → лайки → просмотры.

Данные включают в себя два файла:

shared_articles.csv;
users_interactions.csv.
Начнём работать с файлом shared_articles.csv. Он содержит информацию о статьях, опубликованных на платформе DeskDrop.

Для каждой статьи есть:
дата публикации (временная метка),
исходный URL-адрес,
заголовок,
содержание в виде обычного текста,
язык статьи (португальский — pt или английский — en),
информация о пользователе, который поделился статьёй (автор).
Для временной метки существует два возможных типа событий:

CONTENT SHARED — статья была опубликована на платформе и доступна для пользователей;
CONTENT REMOVED — статья была удалена с платформы и недоступна для дальнейших рекомендаций.
Для простоты мы рассматриваем здесь только тип события CONTENT SHARED.

In [21]:
# Импорт библиотек
from pathlib import Path
from warnings import filterwarnings
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Скрывает INFO и WARNING, оставляет только ERROR
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0' # Отключить oneDNN сообщения

# Автоматическое определение пути к текущему окружению conda
conda_prefix = os.environ.get('CONDA_PREFIX')
if conda_prefix:
    os.environ['XLA_FLAGS'] = f'--xla_gpu_cuda_data_dir={conda_prefix}'

# --- 2. Включение ускорения Intel для CPU --- scikit-learn
try:
    from sklearnex import patch_sklearn, config_context
    patch_sklearn()
    print('Intel Extension для scikit-learn успешно активирован')
except (ImportError, ModuleNotFoundError, Exception) as e:
    print(f'Не удалось активировать Intel Extension для scikit-learn: {e}')
    print(f'Продолжаем работу без ускорения (это нормально)')


from IPython.display import display, Markdown
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

#from arch import arch_model
from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты
from surprise.model_selection import train_test_split
from surprise import SVD, KNNBasic, accuracy
from surprise import BaselineOnly

from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from scipy.linalg import svd
from scipy.optimize import minimize, least_squares


from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k

import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy
import sklearn
import math

import time

from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model


# Если есть TensorFlow, проверяем его:
try:
    import tensorflow as tf
    print(f"TensorFlow видит GPU: {len(tf.config.list_physical_devices('GPU')) > 0}")
except ImportError:
    pass



Intel Extension для scikit-learn успешно активирован
TensorFlow видит GPU: True


Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [2]:
filterwarnings("ignore")
# warnings.filterwarnings('ignore', category=UserWarning)
# warnings.filterwarnings('ignore', message='.*ConvergenceWarning.*')

plt.style.use('seaborn-v0_8') #стиль отрисовки seaborn
sns.set_style("whitegrid")
%matplotlib inline


SEED = 42

# Проверка текущей рабочей директории
print(f"Текущая рабочая директория: {os.getcwd()}")

# Текущая рабочая директория (скорее всего .../IDE)
current_dir = os.getcwd()

# Относительный путь от корня проекта до папки с ноутбуком
# Обратите внимание: используем r'' для корректной обработки слешей и пробелов
relative_path_to_notebook = r'skillfactory/MATH_ML_15'

# Проверяем, не перешли ли мы уже в нужную папку (чтобы не было ошибок при перезапуске ячейки)
if not current_dir.endswith("MATH_ML_15"):
    # Собираем полный путь
    new_dir = os.path.join(current_dir, relative_path_to_notebook)
    
    # Меняем рабочую директорию
    try:
        os.chdir(new_dir)
        print(f"Рабочая директория изменена на: {os.getcwd()}")
    except FileNotFoundError:
        print("Ошибка: путь не найден. Проверьте правильность названий папок.")
else:
    print("Рабочая директория уже верная.")

Текущая рабочая директория: /home/pavel/IDE/skillfactory/MATH_ML_15
Рабочая директория уже верная.


In [3]:
articles_df = pd.read_csv('./data/shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en


In [4]:
interactions_df = pd.read_csv('./data/users_interactions.csv')
interactions_df.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


Теперь откроем второй файл — users_interactions.csv .

Давайте предварительно преобразуем столбцы personId, contentId в таблицах к строкам. Это преобразование пригодится нам в дальнейшем:

```
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)
```

В колонке eventType описаны действия, которые могли совершать пользователи при взаимодействии со статьёй:

VIEW — просмотр,
LIKE — лайк,
COMMENT CREATED — комментарий,
FOLLOW — подписка,
BOOKMARK — добавление в закладки.

В первую очередь нам необходимо понять, как определить, что какая-то статья популярнее других. Если бы из возможных реакций у нас были только лайки или только просмотры, то статьи было бы легко ранжировать в соответствии с этими значениями. Однако у нас есть информация о различных действиях пользователя, и на её основе мы должны создать некий универсальный индекс популярности. Составим его из реакций пользователей, придав им разные веса:
```
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}
```
Веса здесь подобраны исходя из важности каждого действия: оставить комментарий — значит, показать наибольшую вовлечённость, а обычный просмотр, напротив, демонстрирует наименьшую вовлечённость.

In [5]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [6]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}


Создаем признак, который будет отражать числовой вес для взаимодействия со статьёй (в соответствии с приведёнными выше весами). Вычислим среднее значение для полученного признака.

In [7]:
interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

mean_strength = interactions_df['eventStrength'].mean()
print(f"Среднее значение eventStrength: {mean_strength:.2f}")

Среднее значение eventStrength: 1.24


Оставим только тех пользователей, которые взаимодействовали хотя бы с пятью статьями.

In [8]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

Теперь оставим только те взаимодействия, которые касаются только отфильтрованных пользователей (то есть тех, которые взаимодействовали как минимум с пятью статьями). Сколько всего таких взаимодействий?

In [9]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

Сейчас каждое отдельное взаимодействие пользователя со статьёй выделено в отдельную запись, то есть пользователь мог просмотреть статью, лайкнуть и прокомментировать её, и всё это отразилось в трёх действиях. Для удобства соединим все эти действия в некоторый коэффициент, который будет отражать интерес пользователя к статье. Так как каждому возможному действию мы ранее уже присвоили вес, то, по сути, нам нужно просто сложить все действия. Однако полученное число будет увеличиваться с количеством действий, и будет очень большой разброс возможных значений. В таких случаях обычно логарифмируют полученный результат с помощью следующей функции:
``` python 
def smooth_user_preference(x):
    return math.log(1+x, 2)

```

In [10]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].max()
)
        
interactions_full_df = interactions_full_df.reset_index()

Разделим данные на обучающую и тестовую выборки, выбрав в качестве временной отсечки значение 1475519545

In [11]:
from sklearn.model_selection import train_test_split

split_ts = 1475519545
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()


### Задание 6.1

1 point possible (graded)

Найдите оценку взаимодействия пользователя с ID -1032019229384696495 со статьёй с ID 943818026930898372. Результат округлите до двух знаков после точки-разделителя.

Примечание. Здесь и далее (пока не будет указано иное) необходимо работать с обучающей выборкой.

In [12]:
# Target User and Item
target_user = '-1032019229384696495'
target_item = '943818026930898372'

# Find the value
result_row = interactions_train_df[
    (interactions_train_df['personId'] == target_user) & 
    (interactions_train_df['contentId'] == target_item)
]

if not result_row.empty:
    score = result_row['eventStrength'].values[0]
    print(f"Score: {score}")
    print(f"Rounded Score: {score:.2f}")
else:
    print("Interaction not found in training set.")

Score: 2.321928094887362
Rounded Score: 2.32


Теперь давайте попробуем применить memory-based-подход коллаборативной фильтрации.

Примечание. Данных достаточно много, поэтому для увеличения скорости работы преобразуйте таблицу в массив numpy.

### Задание 6.2

1 point possible (graded)

Найдите среднее арифметическое всех чисел в получившемся массиве. Результат округлите до трёх знаков после точки-разделителя.

Теперь давайте попробуем применить memory-based-подход коллаборативной фильтрации.

Примечание. Данных достаточно много, поэтому для увеличения скорости работы преобразуйте таблицу в массив numpy.


In [13]:
### Решение - Задание 6.2

# Создаем матрицу взаимодействий
users_items_pivot_matrix_df = interactions_train_df.pivot(index='personId', 
                                                          columns='contentId', 
                                                          values='eventStrength').fillna(0)

# Преобразуем в массив numpy
ratings_matrix = users_items_pivot_matrix_df.values

# Находим среднее арифметическое
global_mean = np.mean(ratings_matrix)
print(f"Global Mean: {global_mean}")
print(f"Rounded Global Mean: {global_mean:.3f}")

Global Mean: 0.016668620737604056
Rounded Global Mean: 0.017


Перейдём к реализации коллаборативной фильтрации. Ранее мы делали это с помощью библиотеки surprise, однако это не всегда удобно, так как эта библиотека имеет ограниченное количество метрик для оценки качества и небольшой потенциал для более тонкой настройки алгоритма. Поэтому давайте попробуем реализовать алгоритмы коллаборативной фильтрации «с нуля». Такая практика применяется, если необходимо выстроить более сложную систему, чем могут предложить готовые модули. Кроме того, «ручная» реализация алгоритмов позволит лучше понять принцип их работы.

### Задание 6.3

1 point possible (graded)

Постройте матрицу схожести. Для этого вычислите все попарные коэффициенты корреляции для матрицы, полученной в предыдущем задании. Для каждой пары учитывайте только ненулевые значения (так как нулевые обозначают отсутствие взаимодействия и не интересуют нас). Выведите результат, полученный в ячейке с третьим индексом по строкам и сороковым — по столбцам. Ответ округлите до двух знаков после точки-разделителя.


In [14]:
### Решение - Задание 6.3
# For 6.3 we need to consider only non-zero values for correlation.
# We can use pandas corr() which ignores NaNs. 
# So we replace 0 with NaN.
users_items_pivot_nan = users_items_pivot_matrix_df.replace(0, np.nan)

# Calculate User-User similarity (Correlation between rows)
# Pandas corr() computes pairwise correlation of columns.
# To get row correlation (Users), we transpose first.
# Transposed: Columns become Users.
start_time = pd.Timestamp.now()
similarity_matrix = users_items_pivot_nan.T.corr(method='pearson')
end_time = pd.Timestamp.now()
print(f"Correlation calculation took: {end_time - start_time}")

# Shape check
print(f"Similarity Matrix Shape: {similarity_matrix.shape}")

# Get value at index [3, 40]
# Use iloc for integer-location based indexing
value_3_40 = similarity_matrix.iloc[3, 40]
print(f"Similarity at [3, 40]: {value_3_40}")
print(f"Rounded Similarity at [3, 40]: {value_3_40:.2f}")

Correlation calculation took: 0 days 00:00:00.971377
Similarity Matrix Shape: (1112, 1112)
Similarity at [3, 40]: -0.33333333333333326
Rounded Similarity at [3, 40]: -0.33


Теперь у нас есть матрицы схожести пользователей. Их можно использовать для построения рекомендаций. Чтобы это сделать, надо реализовать следующий алгоритм.

Для каждого пользователя:

- Найти пользователей с похожестью больше 0.
- Для каждой статьи вычислить долю пользователей (среди выделенных на первом шаге), которые взаимодействовали со статьёй.
- Порекомендовать статьи (не более 10) с наибольшими долями со второго шага (среди тех, которые пользователь ещё не видел).

### Задание 6.4

1 point possible (graded)

Постройте рекомендательную систему по алгоритму, описанному выше. Найдите первую рекомендацию для строки 35 (если считать с нуля).

In [15]:
### Решение - Задание 6.4
# Get the user ID at index 35 (iloc is purely integer-location based)
# Check if index is valid
if len(users_items_pivot_matrix_df) > 35:
    target_user_id = users_items_pivot_matrix_df.index[35]
    print(f"Target User ID (Index 35): {target_user_id}")

    # Get similarity row for this user
    user_similarities = similarity_matrix.loc[target_user_id]

    # Step 1: Find users with similarity > 0
    similar_users_ids = user_similarities[user_similarities > 0].index
    print(f"Count of users with similarity > 0: {len(similar_users_ids)}")

    # Step 2: Calculate interaction share for each article
    # Subset interactions to similar users
    similar_users_interactions = users_items_pivot_matrix_df.loc[similar_users_ids]

    # Count users who interacted (value > 0)
    article_interaction_counts = (similar_users_interactions > 0).sum()

    # Calculate share
    article_shares = article_interaction_counts / len(similar_users_ids)

    # Step 3: Filter out articles the target user has already seen
    target_user_seen_mask = users_items_pivot_matrix_df.loc[target_user_id] > 0
    seen_articles = target_user_seen_mask[target_user_seen_mask].index
    
    # Drop seen articles
    recommendations = article_shares.drop(index=seen_articles, errors='ignore')

    # Sort descending
    top_recommendations = recommendations.sort_values(ascending=False).head(10)

    print("Top Recommendations:")
    print(top_recommendations)

    if not top_recommendations.empty:
        print(f"Answer (First Recommendation ContentID): {top_recommendations.index[0]}")
else:
    print("Error: DataFrame has fewer than 36 rows.")

Target User ID (Index 35): -174458633445209100
Count of users with similarity > 0: 49
Top Recommendations:
contentId
-5148591903395022444    0.469388
-2447632164766022033    0.387755
4259370161044254504     0.387755
1356221992133852808     0.346939
-8208801367848627943    0.346939
2581138407738454418     0.326531
6437568358552101410     0.306122
-820343972901090172     0.306122
-6783772548752091658    0.306122
-133139342397538859     0.306122
dtype: float64
Answer (First Recommendation ContentID): -5148591903395022444


После того как сделаны предсказания, можно вычислить качество по метрике, которую мы определили в предыдущем модуле при решении этой задачи:
``` python
def calc_precision(column):
    return ( interactions.apply(  lambda row:len(set(row['true_test']).intersection(
                set(row[column]))) /min(len(row['true_test']) + 0.001, 10.0), axis=1)).mean()
```


### Задание 6.5

1 point possible (graded)

Вычислите точность полученного предсказания. Ответ округлите до трёх знаков после точки-разделителя.

In [16]:
### Решение - Задание 6.5
# Prepare test data with true interactions
interactions = interactions_test_df.groupby('personId')['contentId'].agg(set).reset_index()
interactions.rename(columns={'contentId': 'true_test'}, inplace=True)

# Define prediction function
def make_prediction(person_id):
    # If user is not in the training set, we cannot make predictions using this collaborative filtering approach
    if person_id not in users_items_pivot_matrix_df.index:
        return []
    
    # Get similarity row for this user
    if person_id in similarity_matrix.index:
        user_similarities = similarity_matrix.loc[person_id]
    else:
        return []

    # Step 1: Find users with similarity > 0
    similar_users_ids = user_similarities[user_similarities > 0].index
    
    if len(similar_users_ids) == 0:
        return []

    # Step 2: Calculate interaction share for each article
    # Subset interactions to similar users
    similar_users_interactions = users_items_pivot_matrix_df.loc[similar_users_ids]

    # Count users who interacted (value > 0)
    article_interaction_counts = (similar_users_interactions > 0).sum()

    # Calculate share
    article_shares = article_interaction_counts / len(similar_users_ids)

    # Step 3: Filter out articles the target user has already seen
    target_user_seen_mask = users_items_pivot_matrix_df.loc[person_id] > 0
    seen_articles = target_user_seen_mask[target_user_seen_mask].index
    
    # Drop seen articles
    recommendations = article_shares.drop(index=seen_articles, errors='ignore')

    # Sort descending and take top 10
    top_recommendations = recommendations.sort_values(ascending=False).head(10)
    
    return list(top_recommendations.index)

# Apply prediction to all test users
# This might take a little bit of time but N=1112 is small
from tqdm import tqdm
tqdm.pandas()

print("Generating predictions...")
interactions['prediction'] = interactions['personId'].apply(make_prediction)

# Calculate precision
def calc_precision(column, df):
    return (df.apply(lambda row: len(set(row['true_test']).intersection(
                set(row[column]))) / min(len(row['true_test']) + 0.001, 10.0), axis=1)).mean()

precision = calc_precision('prediction', interactions)
print(f"Precision: {precision}")
print(f"Rounded Precision: {precision:.3f}")

Generating predictions...
Precision: 0.004523940705554004
Rounded Precision: 0.005


### Задание 6.6

1 point possible (graded)

Теперь реализуем рекомендательную систему с использованием SVD.

Разложите матрицу взаимодействий пользователей со статьями с помощью функции svd из модуля scipy. Найдите максимальное значение в получившейся матрице U. Результат округлите до двух знаков после точки-разделителя.

In [22]:
### Решение - Задание 6.6
# Using scipy.linalg.svd as requested/implied by the reference "svd(ratings)"
# This performs full SVD on the dense matrix

U, sigma, Vt = svd(ratings_matrix)

print(f"Shape of U: {U.shape}")
max_u = np.max(U)
print(f"Max value in U: {max_u}")
print(f"Rounded Max value in U: {max_u:.2f}")



Shape of U: (1112, 1112)
Max value in U: 0.8585681104884452
Rounded Max value in U: 0.86


Значения матрицы с сингулярными числами отсортированы по убыванию. Допустим, мы хотим оставить только первые 100 компонент и получить скрытые представления размерности 100. Для этого необходимо оставить 100 столбцов в матрице U, только первые 100 значений из sigma (и сделать из них диагональную матрицу) и 100 строк в матрице V. Затем необходимо перемножить преобразованные матрицы.

### Задание 6.7

1 point possible (graded)

Найдите сумму всех элементов в новой сингулярной матрице. Ответ округлите до двух знаков после точки-разделителя.

In [26]:
### Решение - Задание 6.7
k = 100
# Берем первые 100 сингулярных чисел из массива sigma
s = np.diag(sigma[:k])
U_k = U[:, 0:k]
Vt_k = Vt[0:k, :]

sum_sigma_new = np.sum(s)
print(f"Sum of elements in the new singular matrix (top {k} diagonal): {sum_sigma_new}")
print(f"Rounded Sum: {sum_sigma_new:.2f}")

# Reconstruct the matrix (ready for next tasks)
new_ratings_matrix = np.dot(np.dot(U_k, s), Vt_k)
print(f"Reconstructed Matrix Shape: {new_ratings_matrix.shape}")

Sum of elements in the new singular matrix (top 100 diagonal): 2096.432772165698
Rounded Sum: 2096.43
Reconstructed Matrix Shape: (1112, 2366)


Теперь мы можем сделать предсказание по полученной матрице.

Примечание. Помните, что не нужно учитывать статьи, которые уже были просмотрены пользователем.

Найдите для каждого пользователя статьи с наибольшими оценками в восстановленной матрице.

### Задание 6.8

1 point possible (graded)

Вычислите качество полученного предсказания, используя всё ту же метрику точности. Ответ округлите до трёх знаков после точки-разделителя.

In [33]:
### Решение - Задание 6.8
# top_k recommendations (ответ сходится с эталонным только при top_k = 8 или top_k = 7)
top_k = 8

# Wrap reconstructed matrix in DataFrame for easier indexing
new_ratings_df = pd.DataFrame(new_ratings_matrix, 
                              index=users_items_pivot_matrix_df.index, 
                              columns=users_items_pivot_matrix_df.columns)

# Prepare test interactions
interactions = interactions_test_df.groupby('personId')['contentId'].agg(set).reset_index()
interactions.rename(columns={'contentId': 'true_test'}, inplace=True)

def make_prediction_svd(person_id):
    if person_id not in new_ratings_df.index:
        return []
    
    # Get user's predicted scores
    user_scores = new_ratings_df.loc[person_id]
    
    # Identify items user has already seen in training
    seen_mask = users_items_pivot_matrix_df.loc[person_id] > 0
    seen_articles = seen_mask[seen_mask].index
    
    # Exclude seen articles
    user_scores_excluded = user_scores.drop(index=seen_articles, errors='ignore')
    
    # Take top = top_k recommendations
    
    top_recommendations = user_scores_excluded.sort_values(ascending=False).head(top_k)
    
    return list(top_recommendations.index)

# Generate predictions
tqdm.pandas()
print("Generating SVD predictions...")
interactions['prediction_svd'] = interactions['personId'].progress_apply(make_prediction_svd)

# Calculate precision
def calc_precision(column, df):
    return (df.apply(lambda row: len(set(row['true_test']).intersection(
                set(row[column]))) / min(len(row['true_test']) + 0.001, 10.0), axis=1)).mean()

precision_svd = calc_precision('prediction_svd', interactions)
print(f"SVD Precision: {precision_svd}")
print(f"Rounded SVD Precision: {precision_svd:.3f}")

Generating SVD predictions...


  0%|          | 0/982 [00:00<?, ?it/s]

100%|██████████| 982/982 [00:00<00:00, 1906.92it/s]

SVD Precision: 0.0123816552206202
Rounded SVD Precision: 0.012


Эталонный ответ 0,012

Итак, мы реализовали два алгоритма коллаборативной фильтрации буквально с нуля! Теперь для полноты картины давайте реализуем на этих данных гибридную модель и посмотрим, какое качество получится. Для этого воспользуемся уже изученной библиотекой LightFM.



### Задание 6.9

1 point possible (graded)

Возьмите матрицу, подготовленную в задании 6.1. Преобразуйте её в разреженную матрицу:

from scipy.sparse import csr_matrix
ratings_matrix = csr_matrix(ratings)
Воспользовавшись функцией random_train_test_split() из библиотеки lightfm, разделите данные на валидационную и обучающую выборки в соотношении 1:2 (30% на валидационную выборку, 70% на обучающую). В качестве значения параметра random_state возьмите число 13.

Обучите модель LightFM со 100 компонентами, параметром random_state = 13, темпом обучения 0.05 и функцией потерь 'warp'. Обратите внимание на то, что так как в данном случае у нас нет item-признаков, то параметр item_features задавать не нужно.

Вычислите показатель точности (precision@k) при k = 10. Ответ округлите до двух знаков после точки-разделителя.

In [35]:
### Решение - Задание 6.9

from scipy.sparse import csr_matrix
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k

# 1. Sparse Matrix
ratings_matrix_sparse = csr_matrix(users_items_pivot_matrix_df.values)

# 2. Split (30% validation, 70% train)
train_data, test_data = random_train_test_split(ratings_matrix_sparse, test_percentage=0.3, random_state=13)

# 3. Model
model = LightFM(learning_rate=0.05, loss='warp', no_components=100, random_state=13)

# 4. Train (epochs=10 is assumed standard if not specified, often sufficient for exercises)
model.fit(train_data, epochs=10)

# 5. Evaluate
precision_score = precision_at_k(model, test_data, k=10).mean()

print(f"LightFM Precision@10: {precision_score}")
print(f"Rounded LightFM Precision@10: {precision_score:.2f}")

LightFM Precision@10: 0.028684470802545547
Rounded LightFM Precision@10: 0.03


В данном случае модель «из коробки» показала наилучший результат, однако это совсем не показатель того, что стоит пользоваться исключительно готовыми функциями. Зная тонкости работы алгоритмов, вы можете создавать собственные гибридные системы, настраивать отдельные алгоритмы и добиваться ещё лучших результатов.